# To do:

 - Figure out how to signal injuries

In [1]:
import pandas as pd
import numpy as np
import duckdb
import warnings
import math         # haversine_km()
import os

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import warnings
from datetime import datetime, timedelta
from haversine import haversine

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

Today's date: 2026-01-04


In [2]:
%run ./common_utils.ipynb

# Initial Functions

In [3]:
def email(model, error):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA create_Predictions error"
    body = f"Model: {model}_model\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

In [4]:
def create_base_df():
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    for i in [2022, 2023, 2024, 2025]:
        df_actuals = pd.read_csv(f"../tables/{i}/parlay_stats.csv")
        df_actuals['Season'] = i
        df = pd.concat([df, df_actuals])

        df_schd = pd.read_csv(f"../tables/{i}/nba_schedule.csv")
        df_schd['Season'] = i
        df2 = pd.concat([df2, df_schd])

        df_gms = pd.read_csv(f"../tables/{i}/season_gamelogs.csv")
        df_gms['Season'] = i
        df3 = pd.concat([df3, df_gms])

        df_inj = pd.read_csv(f"../tables/{i}/injuries.csv")
        df_inj['Season'] = i
        df4 = pd.concat([df4, df_inj])

    df['Date'] = pd.to_datetime(df.Date)
    df2['Date'] = pd.to_datetime(df2.Date)
    df3['Date'] = pd.to_datetime(df3.Date)
    df3 = df3[~df3[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df4['Date'] = pd.to_datetime(df4.Date)

    df3_temp = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3_temp['PR'] = df3_temp.PTS + df3_temp.REB 
    df3_temp['PA'] = df3_temp.PTS + df3_temp.AST
    df3_temp['RA'] = df3_temp.REB + df3_temp.AST
    df3_temp['PRA'] = df3_temp.PTS + df3_temp.REB + df3_temp.AST
    df3_temp['STL_BLK'] = df3_temp.STL + df3_temp.BLK
    df = df.merge(df3_temp, on=['Season', 'Date', 'Team', 'Player'], how='left')

    df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch['Team_type'] = 'Away'
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch2['Team_type'] = 'Home'
    df_mtch = pd.concat([df_mtch, df_mtch2])
    df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch = df_mtch.sort_values(["Team", "Date"])
    df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
    df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
    df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
    df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
    df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
    df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

    df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
    df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
    df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
    df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)
    
    return df

# Minutes Projection Model

In [5]:
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def travel_km_from_row(row):
    prev = row['PrevLocation']
    cur  = row['Location']
    # missing prev => first game => no travel
    if pd.isna(prev) or pd.isna(cur):
        return 0.0
    # same arena => 0
    if prev == cur:
        return 0.0
    # lookup coords
    prev_coords = arenas.get(prev)
    cur_coords  = arenas.get(cur)
    if not prev_coords or not cur_coords:
        # fallback if code not found
        return 0.0
    return haversine_km(prev_coords[0], prev_coords[1], cur_coords[0], cur_coords[1])

In [7]:
def setup_df_mins(con, df):
    
    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'B2B', 'MP', 'TOV', 'PF', '+/-',
             'Spread', 'Total', 'team_game_num', 'Szn_Wins', 'cup_gm', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP', 'TOV', 'PF', '+/-']:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )
        df[f"{col}_change_L1"] = df[f"{col}_lst_gm"] - df[f"{col}_last_5_avg"]
        df[f"{col}_change_L3"] = df[f"{col}_last_3_avg"] - df[f"{col}_last_10_avg"]
        df[f"{col}_pct_change"] = (
            (df[f"{col}_lst_gm"] - df[f"{col}_last_10_avg"]) /
            (df[f"{col}_last_10_avg"] + 1e-6)
        )
    df["MP_spike"] = (df["MP_lst_gm"] > df["MP_last_10_avg"] + 8).astype(int)
    df["MP_drop"]  = (df["MP_lst_gm"] < df["MP_last_10_avg"] - 8).astype(int)
    df["MP_trend"] = df["MP_last_3_avg"] - df["MP_last_10_avg"]

    games_last_7_days = df.groupby(['Player', 'Season']).rolling('7D', on='Date')['MP'].count().shift(1).to_frame(name='games_last_7_days').reset_index()
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['games_last_7_days'] = df.games_last_7_days.fillna(0).astype(int)
    
    df['prev_team_mins_pct'] = (df.groupby(['Player', 'Season'])['MP'].shift(1)) / 240
           
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
      
    df['missed_games'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']      
          .diff()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    
    df['blowout'] = np.where(abs(df.Spread >= 15), 1, 0)
    df['Injury_Ejection'] = (df['MP'] < 5) | (df['MP_drop'] > 20) | ((df.MP_last_10_avg - df.MP) >= 20)
    
    # Location based features
    df["PrevOpp"] = df.groupby("Player")["Opp"].shift(1)
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    df['Location'] = df.apply(lambda r: r['Team'] if r['Team_type'] == 'Home' else r['Opp'], axis=1)
    df['PrevLocation'] = df.groupby('Player')['Location'].shift(1)
    df['travel_km'] = df.apply(travel_km_from_row, axis=1).fillna(0)
    df['travel_hours'] = df['travel_km'] / 800.0      # approximate flight hours
    df['is_long_trip'] = (df['travel_km'] > 1500).astype(int)
    df['same_arena'] = (df['PrevLocation'] == df['Location']).astype(int)
    
    df = df.drop(['Season', 'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter', 'Szn_Wins', 'TOV', 'PF', '+/-', 
                  'PrevOpp', 'PrevLocation', 'Location'], axis=1)    
    
    return df

# Main Model

In [8]:
def setup_df_main(df, tgt_stat):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TOV']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 'TOV',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP', 'FGA', 'TPA', 'FTA'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

### Today's predictions

In [14]:
def generate_predictions(tgt_stat):
    team_encoder = LabelEncoder()
    player_encoder = LabelEncoder()
    team_type_encoder = LabelEncoder()
    position_encoder = LabelEncoder()
    status_encoder = LabelEncoder()
    
    df_pred = create_base_df()
    
    # Encode string cols
    team_encoder.fit(pd.concat([df_pred["Team"], df_pred["Opp"]], axis=0))
    df_pred["Team"] = team_encoder.transform(df_pred["Team"])
    df_pred["Opp"] = team_encoder.transform(df_pred["Opp"])
    df_pred["Player"] = player_encoder.fit_transform(df_pred["Player"])
    df_pred["Pos"] = position_encoder.fit_transform(df_pred["Pos"])
    df_pred['Team_type'] = team_type_encoder.fit_transform(df_pred['Team_type'])
    df_pred["Status"] = status_encoder.fit_transform(df_pred["Status"])
    
    mins_model = xgb.XGBRegressor()
    mins_model.load_model("../ML_models/mins_model.json")
    stat_model = xgb.XGBRegressor()
    stat_model.load_model(f"../ML_models/{tgt_stat}_model.json")
    
    df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[~(df_lines.Team.isnull())]

    # Predict Mins
    df_lines["Team"] = team_encoder.transform(df_lines["Team"])
    df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
    df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
    df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
    df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
    df_pred_mins = setup_df_mins(con, df_pred)
    df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
    df_pred['MP'] = mins_model.predict(df_pred_mins)

    # Predict Stat
    df_pred = setup_df_main(df_pred, tgt_stat)
    feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
    df_pred = df_pred[df_pred.Date == now][feature_cols]
    df_pred[f"{tgt_stat}_proj"] = stat_model.predict(df_pred)

    df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
    df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
    df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
    df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
    df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

    df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
    df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])

    tds_picks = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]\
                [['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_last_5_avg', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
    tds_picks['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
    tds_picks['Diff2'] = abs((df_pred['MP'] - df_pred['MP_last_5_avg']))
    tds_picks = tds_picks.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
    if tds_picks.shape[0] >= 50:
        print(tds_picks.shape[0], 'rows')
        for tm in tds_picks.Team.unique():
            display(tds_picks[tds_picks.Team == tm])
    else:
        display(tds_picks)
    tds_picks.insert(0, 'Date', pd.to_datetime(now))
    partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

In [15]:
try: 
    generate_predictions('PTS')
except Exception as e:
    email('PTS', e)
    raise Exception(e)

96 rows


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
59,MIL,Giannis Antetokounmpo,PF,SAC,28.793112,27.982349,29.5,20.122019
81,MIL,Myles Turner,C,SAC,27.546740,28.555355,10.5,15.956068
9,MIL,Kevin Porter Jr.,PG,SAC,36.594959,35.855889,17.5,14.395422
12,MIL,Ryan Rollins,PG,SAC,32.635799,29.896870,15.5,17.338411
66,MIL,Bobby Portis,PF,SAC,24.161585,26.135959,12.5,13.530952
67,MIL,Kyle Kuzma,PF,SAC,31.004221,29.381235,11.5,11.970030


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
32,CLE,Sam Merrill,SG,DET,20.592720,26.490156,9.5,18.092762
73,CLE,Evan Mobley,PF,DET,30.644449,30.204794,16.5,12.228904
46,CLE,De'Andre Hunter,SF,DET,26.289061,25.988012,13.5,11.099841
23,CLE,Donovan Mitchell,SG,DET,34.285614,33.217548,28.5,26.715475
31,CLE,Jaylon Tyson,SG,DET,26.475367,27.530558,12.5,13.975595
76,CLE,Dean Wade,PF,DET,14.367634,22.721638,5.5,4.614480
88,CLE,Jarrett Allen,C,DET,27.978958,26.170644,12.5,13.373707
11,CLE,Darius Garland,PG,DET,31.990623,31.930050,17.5,17.915430


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
37,DEN,Christian Braun,SG,BRK,23.772657,27.754787,11.5,3.255720
63,DEN,Aaron Gordon,PF,BRK,21.387344,28.766778,12.5,10.674073
7,DEN,Jamal Murray,PG,BRK,36.494873,36.394264,25.5,23.991871
51,DEN,Peyton Watson,SF,BRK,33.084797,30.811480,14.5,13.580550


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
24,SAC,Zach LaVine,SG,MIL,27.350996,29.441465,16.5,23.654705
71,SAC,Keegan Murray,PF,MIL,35.347092,34.332361,15.5,14.323485
14,SAC,Russell Westbrook,PG,MIL,28.278721,29.423325,14.5,13.496859
64,SAC,DeMar DeRozan,PF,MIL,33.448112,30.823483,19.5,19.035582
90,SAC,Maxime Raynaud,C,MIL,29.210117,26.332997,12.5,12.409883
15,SAC,Dennis Schroder,PG,MIL,26.097654,25.478792,11.5,11.433543


...

,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
4,NOP,Jordan Poole,PG,MIA,27.864929,27.164409,16.5,20.308990
61,NOP,Zion Williamson,PF,MIA,30.086285,28.067300,24.5,27.934052
6,NOP,Jeremiah Fears,PG,MIA,28.162432,26.608550,12.5,10.997179
47,NOP,Trey Murphy III,SF,MIA,29.965057,34.530594,20.5,19.010391
83,NOP,Derik Queen,C,MIA,24.408648,29.976113,12.5,13.535616


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
13,PHO,Collin Gillespie,PG,OKC,31.023195,31.621266,13.5,17.288382
21,PHO,Devin Booker,SG,OKC,31.746786,32.009168,23.5,26.217806
87,PHO,Mark Williams,C,OKC,22.436359,23.466845,10.5,12.414724
77,PHO,Oso Ighodaro,PF,OKC,22.700899,23.958188,5.5,6.561599
56,PHO,Royce O'Neale,SF,OKC,30.079601,28.232986,8.5,9.497964
25,PHO,Grayson Allen,SG,OKC,25.825489,26.125307,11.5,12.261760
49,PHO,Dillon Brooks,SF,OKC,29.964006,29.833093,18.5,18.478537


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
22,OKC,Jalen Williams,SG,PHO,28.980459,28.404469,17.5,14.414038
69,OKC,Chet Holmgren,PF,PHO,30.226877,27.207377,16.5,19.425091
36,OKC,Cason Wallace,SG,PHO,26.460697,25.594665,7.5,9.086094
0,OKC,Shai Gilgeous-Alexander,PG,PHO,34.191216,30.886402,31.5,29.965431
26,OKC,Ajay Mitchell,SG,PHO,26.330261,24.326729,12.5,13.888022
57,OKC,Luguentz Dort,SF,PHO,27.737635,25.562873,6.5,7.073567
30,OKC,Aaron Wiggins,SG,PHO,20.798864,14.218222,9.5,9.243814


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
48,IND,Aaron Nesmith,SF,ORL,28.659945,24.673259,13.5,10.873394
5,IND,Andrew Nembhard,PG,ORL,33.580963,31.811416,18.5,15.955669
43,IND,Ben Sheppard,SG,ORL,19.689177,21.931463,6.5,7.852183
70,IND,Pascal Siakam,PF,ORL,33.972313,31.552694,24.5,24.981583
10,IND,T.J. McConnell,PG,ORL,17.464357,16.140552,9.5,9.689631
93,IND,Jay Huff,C,ORL,19.029228,21.303687,7.5,7.432577


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
40,WAS,Bilal Coulibaly,SG,MIN,28.338726,27.219705,10.5,12.638329
58,WAS,Justin Champagnie,SF,MIN,24.118483,23.214769,9.5,10.676468
35,WAS,Tre Johnson,SG,MIN,24.562168,21.953369,14.5,15.638978
55,WAS,Khris Middleton,SF,MIN,22.777418,22.027139,9.5,9.049287
29,WAS,CJ McCollum,SG,MIN,32.581181,32.002085,18.5,18.115543


In [16]:
try: 
    generate_predictions('PRA')
except Exception as e:
    email('PRA', e)
    raise Exception(e)

75 rows


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
59,MIL,Giannis Antetokounmpo,PF,SAC,28.793112,27.982349,45.5,33.019436
66,MIL,Bobby Portis,PF,SAC,24.161585,26.135959,19.5,24.435452
81,MIL,Myles Turner,C,SAC,27.546740,28.555355,17.5,19.355398
9,MIL,Kevin Porter Jr.,PG,SAC,36.594959,35.855889,29.5,27.942545
12,MIL,Ryan Rollins,PG,SAC,32.635799,29.896870,25.5,26.201416
67,MIL,Kyle Kuzma,PF,SAC,31.004221,29.381235,18.5,18.739780


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
32,CLE,Sam Merrill,SG,DET,20.592720,26.490156,14.5,26.372561
23,CLE,Donovan Mitchell,SG,DET,34.285614,33.217548,38.5,34.010975
46,CLE,De'Andre Hunter,SF,DET,26.289061,25.988012,19.5,15.669240
73,CLE,Evan Mobley,PF,DET,30.644449,30.204794,29.5,25.680750
88,CLE,Jarrett Allen,C,DET,27.978958,26.170644,22.5,25.734476
76,CLE,Dean Wade,PF,DET,14.367634,22.721638,11.5,10.529239
31,CLE,Jaylon Tyson,SG,DET,26.475367,27.530558,20.5,21.173758
11,CLE,Darius Garland,PG,DET,31.990623,31.930050,27.5,27.278900


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
17,DET,Marcus Sasser,PG,CLE,9.722490,8.878393,17.5,9.205511
89,DET,Isaiah Stewart,C,CLE,24.527830,22.905378,21.5,14.963943
50,DET,Ausar Thompson,SF,CLE,27.242525,26.858583,21.5,16.222803
52,DET,Duncan Robinson,SF,CLE,26.681911,27.265386,16.5,13.599176
41,DET,Jaden Ivey,SG,CLE,18.336670,17.495765,15.5,13.323645
2,DET,Cade Cunningham,PG,CLE,36.436550,34.697931,44.5,46.428677


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
1,LAL,Luka Doncic,PG,MEM,35.937141,32.557840,51.5,43.471687
45,LAL,LeBron James,SF,MEM,34.301125,31.004524,35.5,32.345314
38,LAL,Marcus Smart,SG,MEM,30.235998,26.220585,17.5,16.496265


...

,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
84,MIN,Rudy Gobert,C,WAS,31.442608,31.432102,25.5,22.030087
20,MIN,Anthony Edwards,SG,WAS,35.988995,35.433493,39.5,36.694832
28,MIN,Donte DiVincenzo,SG,WAS,31.456375,31.345453,21.5,19.563776
60,MIN,Julius Randle,PF,WAS,34.909481,33.092903,34.5,35.723797
79,MIN,Naz Reid,C,WAS,27.799360,29.060118,24.5,23.335705
65,MIN,Jaden McDaniels,PF,WAS,30.742990,31.086528,20.5,19.515274


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
83,NOP,Derik Queen,C,MIA,24.408648,29.976113,24.5,27.791605
4,NOP,Jordan Poole,PG,MIA,27.864929,27.164409,22.5,24.620111
6,NOP,Jeremiah Fears,PG,MIA,28.162432,26.608550,19.5,17.519009
47,NOP,Trey Murphy III,SF,MIA,29.965057,34.530594,29.5,27.663385
61,NOP,Zion Williamson,PF,MIA,30.086285,28.067300,34.5,36.268932


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
44,BRK,Michael Porter Jr.,SF,DEN,30.237247,32.443921,38.5,35.810040
39,BRK,Terance Mann,SG,DEN,26.431927,26.788549,13.5,12.327315


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
22,OKC,Jalen Williams,SG,PHO,28.980459,28.404469,28.5,25.841024
0,OKC,Shai Gilgeous-Alexander,PG,PHO,34.191216,30.886402,42.5,40.072918
69,OKC,Chet Holmgren,PF,PHO,30.226877,27.207377,27.5,29.862432
26,OKC,Ajay Mitchell,SG,PHO,26.330261,24.326729,18.5,19.544279
36,OKC,Cason Wallace,SG,PHO,26.460697,25.594665,12.5,13.538396


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
54,MIA,Jaime Jaquez Jr.,SF,NOP,28.400702,29.782584,26.5,28.987570
53,MIA,Andrew Wiggins,SF,NOP,30.933498,30.093023,25.5,27.902567
78,MIA,Bam Adebayo,C,NOP,32.556572,30.505844,30.5,29.792009
19,MIA,Norman Powell,SG,NOP,32.496151,30.978225,31.5,30.827520
16,MIA,Davion Mitchell,PG,NOP,31.807127,27.771830,20.5,20.947655
